# HW 1 - Combinatorics

### Please, fill in before you start:

First Name: Julia

Last Name: Mikhailova

Group: 218

In [1]:
import numpy as np
import pandas as pd
import datetime
import time
import random

BASE_SIZE = 100

def date_to_sec(date):
    return time.mktime(datetime.datetime.strptime(date, "%d.%m.%Y").timetuple())


def generate_resources(size=BASE_SIZE):
    data_res = [[i, 'A' + str(i)] for i in range(1, size + 1)]
    return pd.DataFrame(data_res, columns=['ID', 'Name'])


def generate_projects(size=BASE_SIZE):
    # start
    mintime_ts = time.mktime(datetime.datetime.strptime("6.8.2021", "%d.%m.%Y").timetuple())
    maxtime_ts = time.mktime(datetime.datetime.strptime("6.12.2021", "%d.%m.%Y").timetuple())

    RANDOMTIME_start = []
    for _ in range(size):
        random_ts = random.randint(mintime_ts, maxtime_ts)
        RANDOMTIME_start.append(datetime.datetime.fromtimestamp(random_ts).strftime("%d.%m.%Y"))

    # end
    mintime_ts = time.mktime(datetime.datetime.strptime("30.8.2021", "%d.%m.%Y").timetuple())
    maxtime_ts = time.mktime(datetime.datetime.strptime("30.12.2021", "%d.%m.%Y").timetuple())

    RANDOMTIME_end = []
    for _ in range(size):
        random_ts = random.randint(mintime_ts, maxtime_ts)
        RANDOMTIME_end.append(datetime.datetime.fromtimestamp(random_ts).strftime("%d.%m.%Y"))

    # fix cases when start > end
    for i in range(len(RANDOMTIME_start)):
        if date_to_sec(RANDOMTIME_start[i]) > date_to_sec(RANDOMTIME_end[i]):
            RANDOMTIME_start[i], RANDOMTIME_end[i] = RANDOMTIME_end[i], RANDOMTIME_start[i]

    data_proj = [[i + 1, 'z' + str(i + 1), RANDOMTIME_start[i], RANDOMTIME_end[i]] for i in range(size)]

    return pd.DataFrame(data_proj, columns=['ID', 'Name', 'Start Time', 'End Time'])



def generate_expertise(coef, df_resources, df_projects):
    size = len(df_resources) * coef

    res_ID_ = list(np.random.choice(df_resources['ID'], size, replace=True))
    proj_ID_ = list(np.random.choice(df_projects['ID'], size, replace=True))

    random.shuffle(res_ID_)
    random.shuffle(proj_ID_)

    data_exper = [[i + 1, res_ID_[i], proj_ID_[i]] for i in range(size)]

    return pd.DataFrame(data_exper, columns=['ID', 'ID_res', 'ID_pro'])


def generate_all_data(size=BASE_SIZE, coef=5):
    df_resources = generate_resources(size)
    df_projects = generate_projects(size)
    df_expertise = generate_expertise(coef, df_resources, df_projects)
    return df_resources, df_projects, df_expertise

In [2]:
# generate data, size can be changed
df_resources, df_projects, df_expertise = generate_all_data()

In [3]:
# fix cases when there are projects without resources

for id_ in df_projects['ID']:
    if not any(df_expertise['ID_pro'] == id_):
        cur_res = np.random.choice(df_resources['ID'], 1)[0]
        new_expertise = pd.DataFrame([[len(df_expertise) + 1, cur_res, id_]], columns=['ID', 'ID_res', 'ID_pro'])
        df_expertise = pd.concat([df_expertise, new_expertise], ignore_index=True)

df_expertise

,ID,ID_res,ID_pro
0,1,31,5
1,2,97,14
2,3,1,69
3,4,70,37
4,5,89,69
...,...,...,...
495,496,82,65
496,497,96,89
497,498,10,68
498,499,88,23


In [4]:
feasible_solutions = {}

for project_index, project_rows in df_projects.iterrows():
    project_id = project_rows['ID']
    res_ids = df_expertise[df_expertise['ID_pro'] == project_id]['ID_res']
    feasible_solutions[project_id] = []

    for res_id in res_ids:
        feasible_solutions[project_id].append(res_id)

feasible_solutions

{1: [73, 9, 33],
 2: [13, 42, 28, 84, 26, 51, 90, 25, 83, 59, 14, 62, 88, 89],
 3: [95, 73, 2, 39, 16, 92],
 4: [3, 68, 1, 74, 94],
 5: [31, 86, 39, 56, 25, 30],
 6: [61, 45, 35, 16, 10, 51],
 7: [82, 100, 83, 68],
 8: [23, 52, 75, 72, 79],
 9: [80, 2, 61],
 10: [20, 89, 94],
 11: [34, 5],
 12: [35, 65, 4, 7, 87],
 13: [32, 96, 36, 49, 66],
 14: [97, 33, 36, 45, 84, 91, 1],
 15: [53, 56, 90, 13, 76, 21, 88, 47],
 16: [80, 96, 76, 10, 55],
 17: [45, 28, 19, 65, 13, 89, 1],
 18: [75, 39, 45, 67, 20, 5],
 19: [18, 31, 75],
 20: [81, 67, 52, 21, 24],
 21: [16, 25, 41, 70, 83],
 22: [8, 38],
 23: [72, 13, 68, 100, 49, 51, 66, 88],
 24: [52, 86, 15, 73, 56],
 25: [90, 91, 65, 44, 57, 86],
 26: [57, 4, 20, 91, 23, 92],
 27: [46, 66, 9, 100, 54, 85, 1],
 28: [60, 23],
 29: [37, 20, 56],
 30: [53, 87, 78, 96, 85, 12, 38, 20],
 31: [14, 53, 21, 46],
 32: [41, 53, 86, 29, 39, 13, 75, 65],
 33: [58, 4],
 34: [53, 27, 37, 47],
 35: [29, 94, 54, 15],
 36: [10, 83, 76],
 37: [70, 3, 83, 12, 87, 41],


In [5]:
import time
import datetime

# keys = Project IDs df_expertise
keys = list(feasible_solutions.keys())

def is_valid_solution(solution):

    for i in range(len(keys)-1):
        for j in range(i+1, len(keys)):

            if solution[i] == solution[j]:
                start_i = date_to_sec(df_projects["Start Time"][i])
                end_i = date_to_sec(df_projects["End Time"][i])

                start_j = date_to_sec(df_projects["Start Time"][j])
                end_j = date_to_sec(df_projects["End Time"][j])

                if start_i <= start_j:
                    if end_i >= start_j:
                        return False

                elif start_i >= start_j:
                    if end_j >= start_i:
                        return False

    return True

In [6]:
def find_feasible_solution(feasible_solutions):
    keys = list(feasible_solutions.keys())
    stack = [(0, [])]
    solution = None

    while stack:
        depth, current_solution = stack.pop()
        if depth >= len(keys):
            if is_valid_solution(current_solution):
                solution = current_solution
                break
        else:
            current_key = keys[depth]
            for value in feasible_solutions[current_key]:
                next_solution = current_solution + [str(value)]
                stack.append((depth + 1, next_solution))

    return solution

found_solution = find_feasible_solution(feasible_solutions)
found_solution

KeyboardInterrupt: 